# Faire des statistiques


In [ ]:
import pandas as pd
df = pd.read_csv("../../data/CSS_exact_openalex.csv", low_memory=False)

Deux variables :
- dépendante : nombre de citations `cited_by_count`
- indépendantes : années ; langue ; nombre auteurs, language `language`, open accès `open_access.is_oa`, type `type`

In [ ]:
df["authorships.raw_author_name"].isna().sum()

Traitement des données

In [ ]:
# enlever les lignes sans auteurs
df = df[~df["authorships.raw_author_name"].isna()]

# ajouter le nombre d'auteurs
df["nb_auteurs"] = df["authorships.raw_author_name"].apply(lambda x: len(x.split("|")))

# langue anglais
df = df[df["language"] == "en"]

# renommer les colonnes
df = df.rename(columns={"open_access.is_oa":"openaccess"})

# numérique pour des variables
df["openaccess"] = df["openaccess"].astype(int)

## Univarié

Indicateurs

Distribution

recodage

## Bivarié

Quanti/quanti .corr

In [ ]:
from scipy.stats import pearsonr



Quanti/quali

.groupby

In [ ]:
from scipy.stats import f_oneway

Possibilité d'utiliser d'autres bibliothèques, comme pinguin https://pingouin-stats.org/build/html/index.html

In [ ]:
#pip install pingouin

Quali/Quali

pd.crosstab

In [ ]:
from scipy.stats import chi2_contingency

## Modèles

### Régression linéaire

In [ ]:
import statsmodels.api as sm


X = df[['publication_year', 'retracted', 'nb_auteurs']] 
X = sm.add_constant(X) 
y = df["cited_by_count"]

model = sm.OLS(y, X).fit()


print(model.summary())

#### Régression logistique

Quels sont les prédicteurs du fait d'avoir plus de 10 citations ?

Création d'une variable 0 (pas du tout ou rarement) et 1 sinon

In [ ]:
df["citations10"] = df["cited_by_count"] > 10

Avec Scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
df["open"]

In [ ]:
X = pd.get_dummies(df[["publication_year", "openaccess"]], drop_first=True)
y = df["citations10"]

In [ ]:
modele = LogisticRegression()
modele.fit(X, y)
modele.coef_

Avec Statsmodels

In [ ]:
import statsmodels.api as sm
from patsy import dmatrices

In [ ]:
y,X = dmatrices('citations10 ~ C(openaccess) + publication_year', 
                data = df, 
                return_type = 'dataframe')

In [ ]:
modele = sm.Logit(y["citations10[True]"],X)
resultat = modele.fit()
resultat.summary()

Avec la version toute intégrée

In [ ]:
#!pip install -U pyshs

In [ ]:
import pyshs

In [ ]:
pyshs.regression_logistique(df, "citations10", ["openaccess", "publication_year"],)